In [30]:
from vertexai import rag
from vertexai.generative_models import GenerativeModel, Tool
import vertexai

In [31]:
PROJECT_ID = "formidable-feat-466408-r6"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [32]:
# Create RagCorpus
# Configure embedding model, for example "text-embedding-005".
embedding_model_config = rag.RagEmbeddingModelConfig(
    vertex_prediction_endpoint=rag.VertexPredictionEndpoint(
        publisher_model="publishers/google/models/text-embedding-005"
    )
)

In [33]:
display_name="all-marathi-books"

rag_corpus = rag.create_corpus(
    display_name=display_name,
    backend_config=rag.RagVectorDbConfig(
        rag_embedding_model_config=embedding_model_config
    ),
)

In [34]:
rag_corpus.name

'projects/44474009687/locations/us-central1/ragCorpora/4749045807062188032'

In [35]:
paths = ["gs://agentic_ai_ebooks_bucket/Books"]
# paths = ["https://drive.google.com/drive/folders/classroom_books"]

In [36]:
# Import Files to the RagCorpus
import time

start_time = time.time()

rag.import_files(
    rag_corpus.name,
    paths,
    # Optional
    transformation_config=rag.TransformationConfig(
        chunking_config=rag.ChunkingConfig(
            chunk_size=512,
            chunk_overlap=100,
        ),
    ),
    max_embedding_requests_per_min=1000,  # Optional
)

end_time = time.time()

print("Total time to build the indexing", end_time-start_time)

Total time to build the indexing 22.51996397972107


In [37]:
rag.list_files(rag_corpus.name)

ListRagFilesPager<>

In [12]:
# Direct context retrieval
rag_retrieval_config = rag.RagRetrievalConfig(
    top_k=3,  # Optional
    filter=rag.Filter(vector_distance_threshold=0.5),  # Optional
)
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
            # Optional: supply IDs from `rag.list_files()`.
            # rag_file_ids=["rag-file-1", "rag-file-2", ...],
        )
    ],
    text="What is RAG and why it is helpful?",
    rag_retrieval_config=rag_retrieval_config,
)
# print(response)

contexts {
  contexts {
    source_uri: "gs://agentic_ai_ebooks_bucket/classroom_books/balbharati_part_1.pdf"
    text: "\340\244\256\340\245\213\340\244\234 \340\244\265 \340\244\270\340\244\276\340\244\202\340\244\227. Count and tell.\r\n\340\244\265\340\244\260\340\245\215\340\244\227\340\245\200\340\244\225\340\244\260\340\244\243 \340\244\225\340\244\260.\r\nClassify.\r\n\340\244\265\340\244\276\340\244\232. Read.\r\n\340\244\270\340\244\202\340\244\265\340\244\276\340\244\246 \340\244\270\340\244\276\340\244\247. Converse.\r\n\340\244\232\340\244\277\340\244\225\340\244\277\340\244\244\340\245\215\340\244\270\340\244\225 \340\244\265\340\244\277\340\244\232\340\244\276\340\244\260 \340\244\225\340\244\260.\r\nThink critically.\r\n\340\244\252\340\245\201\340\244\242\340\245\200\340\244\262 \340\244\270\340\244\202\340\244\254\340\245\213\340\244\247\340\244\276\340\244\225\340\244\241\340\245\207\r\nTowards the next concept.\r\n\340\244\254\340\244\230 \340\244\265 \340\244\225\3

In [4]:
# Enhance generation
# Create a RAG retrieval tool

# Direct context retrieval
rag_retrieval_config = rag.RagRetrievalConfig(
    top_k=3,  # Optional
    filter=rag.Filter(vector_distance_threshold=0.5),  # Optional
)


rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus='projects/44474009687/locations/us-central1/ragCorpora/6917529027641081856',  # Currently only 1 corpus is allowed.
                    # Optional: supply IDs from `rag.list_files()`.
                    # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                )
            ],
            rag_retrieval_config=rag_retrieval_config,
        ),
    )
)

In [5]:
# Create a Gemini model instance
rag_model = GenerativeModel(
    model_name="gemini-2.0-flash-001", tools=[rag_retrieval_tool]
)

# Generate response
response = rag_model.generate_content("What are the 21st century skills")
print(response.text)

/opt/conda/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


21 व्या शतकातील कौशल्ये:

* चिकित्सक विचार (Critical Thinking)
* माहिती (Information)
* लवचीकता (Flexibility)
* नेतृत्व (Leadership)
* पुढाकार (Initiative)
* उत्पादकता (Productivity)
* सामाजिक कौशल्ये (Social Skills)
* माध्यम (Media)
* तंत्रज्ञान (Technology)
* सर्जनशीलता (Creativity)
* सहकार्य (Collaboration)
* संप्रेषण (Communication)
* अध्ययन कौशल्ये (Learning Skills)
* साक्षरता कौशल्ये (Literacy Skills)
* जीवन कौशल्ये (Life Skills)



In [16]:
# Generate response
response = rag_model.generate_content("Help me to summarize the ala paus ala poem")
print(response.text)

मला माफ करा, तुमच्या क्वेरीमध्ये 'आला पाऊस आला' या कवितेबद्दल कोणतीही माहिती उपलब्ध नाही.



In [6]:
# Generate response
response = rag_model.generate_content("Help me to prepare a content for class 1 from balbharati part 1 for the first week of the school")
print(response.text)

Based on the provided text from Balbharati Part 1 for Class 1, the first week of school could include:

*   **Activities**: Drawing pictures, singing songs, indoor and outdoor games, acting, listening and saying, doing activities.
*   **Skills**: Hand and foot coordination, acting and drama through daily actions, presentation of physical and verbal acting.
*   **Concepts**: Introduction of emojis used in the textbook.
*   **Actions**: Observation, counting, classifying, reading, conversing, critical thinking, looking and doing, writing, filling colors, discussing, solving puzzles.


In [17]:
# Generate response
response = rag_model.generate_content("Help me to summarize the ala paus ala")
print(response.text)

मला माफ करा, 'ala paus ala' बद्दल माहिती देण्यासाठी माझ्याकडे पुरेसा डेटा नाही.


In [18]:
# Generate response
response = rag_model.generate_content("Help me to summarize the Mazya ya otivar")
print(response.text)

मला माफ करा, 'Mazya ya otivar' बद्दल कोणतीही माहिती उपलब्ध नाही. तथापि, "मीनूची सहल" नावाच्या एका कथेमध्ये, ज्युली, हमीद आणि जोसेफ नावाचे मित्र मीनूला बागेत घेऊन जाण्यासाठी तिच्या घरी गेले. ते खूप चांगले मित्र होते आणि त्यांनी रविवारी बागेत खेळायला जायचे ठरवले.



In [19]:
# Generate response
response = rag_model.generate_content("What are the six thinking hats")
print(response.text)

सहा थिंकिंग हॅट्स (Six thinking hats) चा वापर विद्यार्थ्यांच्या विचारांना आणि कल्पनांना चालना मिळणार आहे. त्यामुळे इयत्ता पहिलीपासूनच विद्यार्थ्यांमध्ये चिकित्सक विचार, विश्लेषणात्मक विचार, तार्किक विचार, वैज्ञानिक दृष्टिकोन इत्यादी विकसित होण्यास मदत होणार आहे.



In [21]:
# Generate response
response = rag_model.generate_content("Help me to summarize the gadi ali gadi ali poem")
print(response.text)

मला माफ करा, "गाडी आली गाडी आली" या कवितेबद्दल कोणतीही माहिती उपलब्ध नाही. तथापि, " balancebharti_part_4.pdf " या दस्तऐवजात एक कविता आहे ज्यामध्ये माणसाने पंख नसतानाही विमानातून आकाशात झेप घेतली आहे आणि देश-विदेशात फिरत आहे. तसेच, माणूस जमीन, पाणी आणि आकाशात सहजपणे फिरू शकतो आणि अंतराळयानातून अवकाशातही जाऊ शकतो, असा उल्लेख आहे.



In [23]:
# Generate response
response = rag_model.generate_content("I am teacher who teaches for the 1st standard students. Help me to generate the content for the first month of the school")
print(response.text)

इयत्ता पहिलीच्या पहिल्या महिन्यासाठी शैक्षणिक साहित्य:

*   विद्यार्थी हात व पायांच्या लयबद्ध हालचाली करतील, बडबडगीत व समूहगीतावर अभिनयनृत्य करतील.
*   रोजच्या व्यवहारातील कृतींच्या मदतीने अभिनय व नाटकाची ओळख करून घेणे.
*   कायिक व वाचिक अभिनय सादर करणे.
*   पाठ्यपुस्तकातील चिन्हे (emojis) - चित्र काढ, गाणे गा, बैठे खेळ, निरीक्षण कर, मोज व सांग, वर्गीकरण कर, वाच, संवाद साध, चिकित्सक विचार कर.


In [24]:
# Generate response
response = rag_model.generate_content("I am teacher who teaches for the 1st standard students. Help me to generate the quiz for the first chapter of the part 1 book")
print(response.text)

मला माफ करा, या क्वेरीमध्ये मदत करण्यासाठी पुरेसा संदर्भ माझ्याकडे नाही. माझ्याकडे इयत्ता पहिली भाग-१ च्या पहिल्या अध्यायाबद्दल कोणतीही माहिती नाही.

